In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

from matplotlib import pyplot as plt
from sqlalchemy import create_engine

from ppcollapse import setup_logger
from ppcollapse.utils.config import ConfigManager
from ppcollapse.utils.database import get_collapses_df, get_image
from setup_django_ppcx import setup_django

# Setup Django
config = ConfigManager(config_path="config.yaml")
setup_django(db_config=config.get("database"))

logger = setup_logger(level="INFO", name="ppcx")
config = ConfigManager(config_path="config.yaml")
db_engine = create_engine(config.db_url)

In [ ]:
engine = create_engine(config.db_url)
df = get_collapses_df(engine)
if df.empty:
    logger.warning("No collapses found in database.")
df

In [ ]:
from shapely import wkt as shapely_wkt
from shapely.ops import unary_union

# Example: merge multiple polygons from your collapse dataframe

collapse_1 = df.iloc[0]
collapse_2 = df.iloc[1]

geom1 = shapely_wkt.loads(collapse_1["geom_wkt"])
geom2 = shapely_wkt.loads(collapse_2["geom_wkt"])

# Merge into single geometry (will be MultiPolygon if not contiguous)
merged_geom = unary_union([geom1, geom2])

# If they ARE contiguous, this will return a Polygon
# If they're NOT contiguous, this will return a MultiPolygon
print(type(merged_geom))  # Polygon or MultiPolygon


In [ ]:
image = get_image(image_id=int(collapse_1["image_id"]), config=config)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image)
# Handle both Polygon and MultiPolygon
if merged_geom.geom_type == "Polygon":
    # Single polygon - plot directly
    xs, ys = merged_geom.exterior.xy
    ax.plot(xs, ys, color="red", linewidth=2)
    ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
elif merged_geom.geom_type == "MultiPolygon":
    # Multiple polygons - iterate and plot each
    for poly in merged_geom.geoms:
        xs, ys = poly.exterior.xy
        ax.plot(xs, ys, color="red", linewidth=2)
        ax.fill(xs, ys, facecolor="red", edgecolor="none", alpha=0.3)
else:
    raise ValueError("Merged geometry is neither Polygon nor MultiPolygon.")
ax.set_axis_off()
ax.set_title("Collapse Geometry", fontsize=10, pad=5)

In [ ]:
# Now import Django models
from django.contrib.gis.geos import GEOSGeometry
from ppcx_app.models import Collapse
from shapely import wkt as shapely_wkt


In [ ]:
# Get the merged geometry
merged_geom = unary_union([geom1, geom2])

# Convert Shapely geometry to Django GEOSGeometry
wkt_str = shapely_wkt.dumps(merged_geom)
django_geom = GEOSGeometry(wkt_str, srid=0)

collapse_obj = Collapse.objects.get(id=collapse_1["id"])
collapse_obj.geom = django_geom